In [1]:
!pip install customtkinter fpdf2


   ---------------------------------------- 0/3 [fpdf2]
   ---------------------------------------- 0/3 [fpdf2]
   ---------------------------------------- 0/3 [fpdf2]
   ---------------------------------------- 0/3 [fpdf2]
   -------------------------- ------------- 2/3 [customtkinter]
   -------------------------- ------------- 2/3 [customtkinter]
   -------------------------- ------------- 2/3 [customtkinter]
   ---------------------------------------- 3/3 [customtkinter]



In [5]:
import os
import csv
from tkinter import ttk, messagebox
import threading

from fpdf import FPDF, XPos, YPos

import customtkinter as ctk
from fpdf import FPDF


FILE_NAME = "students_jupyter.csv"

# Ensure CSV exists with header
if not os.path.exists(FILE_NAME):
    with open(FILE_NAME, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Roll", "Name", "M1", "M2", "M3", "Total", "Percentage", "Grade"])


# ------------------------------
# Utility Functions
# ------------------------------
def calc(m1, m2, m3):
    m1, m2, m3 = int(m1), int(m2), int(m3)
    total = m1 + m2 + m3
    pct = round(total / 300 * 100, 2)

    if pct >= 90: grade = "A+"
    elif pct >= 80: grade = "A"
    elif pct >= 70: grade = "B"
    elif pct >= 60: grade = "C"
    elif pct >= 50: grade = "D"
    else: grade = "F"

    return total, pct, grade


def read_all():
    rows = []
    with open(FILE_NAME, "r") as f:
        reader = csv.reader(f)
        next(reader)
        for r in reader:
            rows.append(r)
    return rows


def overwrite(rows):
    with open(FILE_NAME, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Roll", "Name", "M1", "M2", "M3", "Total", "Percentage", "Grade"])
        writer.writerows(rows)


# ------------------------------
# PDF Export
# ------------------------------
def export_pdf(student):
    roll, name, m1, m2, m3, total, pct, grade = student
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=14)

    pdf.cell(0, 10, "Report Card", ln=True, align="C")
    pdf.ln(10)

    pdf.set_font("Arial", size=12)
    pdf.cell(0, 8, f"Roll No: {roll}", ln=True)
    pdf.cell(0, 8, f"Name: {name}", ln=True)
    pdf.cell(0, 8, f"Marks 1: {m1}", ln=True)
    pdf.cell(0, 8, f"Marks 2: {m2}", ln=True)
    pdf.cell(0, 8, f"Marks 3: {m3}", ln=True)
    pdf.cell(0, 8, f"Total: {total}", ln=True)
    pdf.cell(0, 8, f"Percentage: {pct}%", ln=True)
    pdf.cell(0, 8, f"Grade: {grade}", ln=True)

    filename = f"Report_{roll}.pdf"
    pdf.output(filename)

    messagebox.showinfo("Exported", f"PDF saved: {filename}")


# ------------------------------
# GUI Start Function (Thread)
# ------------------------------
def start_gui():
    ctk.set_appearance_mode("System")
    ctk.set_default_color_theme("blue")

    app = ctk.CTk()
    app.title("Student Manager (Jupyter Compatible)")
    app.geometry("900x600")

    #---------------------- Left Frame ----------------------
    left = ctk.CTkFrame(app)
    left.pack(side="left", fill="y", padx=10, pady=10)

    ctk.CTkLabel(left, text="Add Student", font=("Arial", 18, "bold")).pack(pady=10)

    entry_roll = ctk.CTkEntry(left, placeholder_text="Roll No")
    entry_roll.pack(pady=5, padx=10)

    entry_name = ctk.CTkEntry(left, placeholder_text="Name")
    entry_name.pack(pady=5, padx=10)

    entry_m1 = ctk.CTkEntry(left, placeholder_text="Marks 1")
    entry_m1.pack(pady=5, padx=10)

    entry_m2 = ctk.CTkEntry(left, placeholder_text="Marks 2")
    entry_m2.pack(pady=5, padx=10)

    entry_m3 = ctk.CTkEntry(left, placeholder_text="Marks 3")
    entry_m3.pack(pady=5, padx=10)

    #---------------------- Table ----------------------
    right = ctk.CTkFrame(app)
    right.pack(side="right", fill="both", expand=True, padx=10, pady=10)

    columns = ["Roll", "Name", "M1", "M2", "M3", "Total", "Pct", "Grade"]
    tree = ttk.Treeview(right, columns=columns, show="headings")

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)
    tree.pack(fill="both", expand=True)

    def refresh():
        for row in tree.get_children():
            tree.delete(row)
        for r in read_all():
            tree.insert("", "end", values=r)

    #---------------------- Add Student ----------------------
    def add_student():
        roll = entry_roll.get()
        name = entry_name.get()
        m1 = entry_m1.get()
        m2 = entry_m2.get()
        m3 = entry_m3.get()

        if not (roll and name and m1 and m2 and m3):
            messagebox.showerror("Error", "All fields are required")
            return

        total, pct, grade = calc(m1, m2, m3)

        row = [roll, name, m1, m2, m3, total, pct, grade]

        with open(FILE_NAME, "a", newline="") as f:
            csv.writer(f).writerow(row)

        refresh()
        messagebox.showinfo("Added", "Student added successfully")

    ctk.CTkButton(left, text="Add Student", command=add_student).pack(pady=10)

    #---------------------- Delete ----------------------
    def delete_student():
        sel = tree.focus()
        if not sel:
            messagebox.showerror("Error", "Select a row first")
            return
        roll = tree.item(sel)["values"][0]
        rows = read_all()
        rows = [r for r in rows if r[0] != roll]
        overwrite(rows)
        refresh()
        messagebox.showinfo("Deleted", "Student deleted")

    ctk.CTkButton(left, text="Delete Selected", fg_color="red", command=delete_student).pack(pady=5)

    #---------------------- Export PDF ----------------------
    def export_selected():
        sel = tree.focus()
        if not sel:
            messagebox.showerror("Error", "Select a row first")
            return
        export_pdf(tree.item(sel)["values"])

    ctk.CTkButton(left, text="Export PDF", fg_color="green", command=export_selected).pack(pady=5)

    refresh()
    app.mainloop()


# Run GUI in a thread (important for Jupyter)
threading.Thread(target=start_gui).start()


C:\Users\mayur\AppData\Local\Temp\ipykernel_2984\2024305644.py:63: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=14)
C:\Users\mayur\AppData\Local\Temp\ipykernel_2984\2024305644.py:65: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "Report Card", ln=True, align="C")
C:\Users\mayur\AppData\Local\Temp\ipykernel_2984\2024305644.py:68: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
C:\Users\mayur\AppData\Local\Temp\ipykernel_2984\2024305644.py:69: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 8, f"Roll No: {roll}", ln=True)
C:\Users\mayur\AppData\Local\Temp\ipykernel_2984\2024305644